In [3]:
import cv2
from keras.models import load_model
from mtcnn.mtcnn import MTCNN
import numpy as np
from face_recognition import compare_faces, face_distance
from sklearn.svm import SVC

Using TensorFlow backend.
/home/ketan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ketan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ketan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ketan/.local/lib/python3.6/site-packages/tensorflow/python/fra

In [4]:
class detection:
    def __init__(self):
        print("loading facenet...", end = '')
        self.embeddings = load_model('/home/ketan/Hackathons/facenet_keras.h5')
        print("...|")
        print("Loading MTCNN...", end = '')
        self.face_detector = MTCNN()
        print("...|")
        self.tolerance = 0.4
        self.wanted = []
        
    def update_wanted(self, image):
        print("updating Wanted List...")
        faces, coordinates = self.find_faces(image)
        #print(faces)
        encoding = self.find_embeddings(faces)
        #print('encoding: ', encoding)
        self.wanted.append(encoding[0])
        print("wanted updation completed")
        
    def find_faces(self, image):
        print("Detecting Face...", end = '\t')
        faces = self.face_detector.detect_faces(image)
        if len(faces) == 0:
            return np.zeros((160, 160, 3)), None
        face_images = []
        coordinates = []
        for face in faces:
            [x, y, w, h] = face['box']
            coordinates.append(face['box'])
            face_images.append(cv2.resize(image[y:y+h, x:x+w], (160, 160)))
        print("completed")
        return face_images, coordinates

    def find_embeddings(self, list_faces):
        print("Calculating Embeddings...", end = '\t')
        embedding_list = []
        if len(list_faces) == 0:
            print("no face detected!!!")
            return embedding_list
        for face in list_faces:
            mean, std = face.mean(), face.std()
            face = (face - mean) / std
            samples = np.expand_dims(face, axis=0)
            try:
                embedding_list.append(self.embeddings.predict(samples))
            except ValueError:
                print("object in front of face.")
        print("completed")
        return embedding_list
    
    def compare(self, face_embeddings):
        print("compairing...")
        distances = []
        for wanted_index, wanted in enumerate(self.wanted):
            for index, face in enumerate(face_embeddings):
                temp_dist = face_distance(face, wanted)
                print("compairing wanted"+str(wanted_index)+" and face"+str(index)+" and result:", temp_dist)
                if temp_dist <= self.tolerance:
                    print("!!!!!!!!!!!!!!!!!!!!match found!!!!!!!!!!!!!!!!!!!!")
                    # wanted found
                    # wanted_found(wanted_index)
                distances.append(temp_dist)
#                 for tolerance in range(1,9,1):
#                     temp = compare_faces(face, wanted, tolerance = float(tolerance))
#                     print("compairing wanted"+str(wanted_index)+" and face"+str(index)+"for tolerance"+str(tolerance)+" and result:", temp)
#                     if result == True:
#                         pass
#                         # run updation code
#                     result.append(temp)
        print("compairing completed")

In [5]:
print("creating face detector...")
d = detection()
print('detector created successfully.')

# Adding wanted
cam = cv2.VideoCapture(0)
while(1):
    ok, frame = cam.read()
    if ok:
        cv2.imshow("Live Image.", frame)
        k = cv2.waitKey(0)
        if k & 0xFF == ord('q'):
            d.update_wanted(frame)
            cv2.destroyAllWindows()
            cam.release()
            break
        

W0922 01:55:22.480965 140491657791296 deprecation_wrapper.py:119] From /home/ketan/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0922 01:55:22.569778 140491657791296 deprecation_wrapper.py:119] From /home/ketan/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0922 01:55:22.596227 140491657791296 deprecation_wrapper.py:119] From /home/ketan/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0922 01:55:22.597755 140491657791296 deprecation_wrapper.py:119] From /home/ketan/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.


creating face detector...
loading facenet...

W0922 01:55:23.237978 140491657791296 deprecation_wrapper.py:119] From /home/ketan/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0922 01:55:39.766572 140491657791296 deprecation.py:506] From /home/ketan/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
/home/ketan/.local/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
W0922 01:55:50.704947 140491657791296 deprecation.py:506] From /home/ketan/.local/lib/python3.6/site-packages/tensorflow/pyt

...|
Loading MTCNN...

W0922 01:55:50.906355 140491657791296 deprecation.py:323] From /home/ketan/.local/lib/python3.6/site-packages/mtcnn/layer_factory.py:221: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
W0922 01:55:51.405087 140491657791296 deprecation_wrapper.py:119] From /home/ketan/.local/lib/python3.6/site-packages/mtcnn/layer_factory.py:196: The name tf.nn.xw_plus_b is deprecated. Please use tf.compat.v1.nn.xw_plus_b instead.



...|
detector created successfully.
updating Wanted List...
Detecting Face...	completed
Calculating Embeddings...	completed
wanted updation completed


In [6]:
cam = cv2.VideoCapture(0)

while 1:
    ok, frame = cam.read()
    if ok:
        faces, coordinated = d.find_faces(frame)
        encodings = d.find_embeddings(faces)
        d.compare(encodings)
        cv2.imshow("live image", frame)
        for index, face in enumerate(faces):
            cv2.imshow("face "+str(index), face)
        k = cv2.waitKey(2)
        if k & 0xff == ord('q'):
            cv2.destroyAllWindows()
            cam.release()
            break
    else:
        print('camera not working')

Detecting Face...	completed
Calculating Embeddings...	completed
compairing...
compairing wanted0 and face0 and result: [6.36907]
compairing completed
Detecting Face...	completed
Calculating Embeddings...	completed
compairing...
compairing wanted0 and face0 and result: [5.482408]
compairing completed
Detecting Face...	completed
Calculating Embeddings...	completed
compairing...
compairing wanted0 and face0 and result: [6.7021575]
compairing completed
Detecting Face...	completed
Calculating Embeddings...	completed
compairing...
compairing wanted0 and face0 and result: [6.3410544]
compairing completed
Detecting Face...	completed
Calculating Embeddings...	completed
compairing...
compairing wanted0 and face0 and result: [6.683327]
compairing completed
Detecting Face...	completed
Calculating Embeddings...	completed
compairing...
compairing wanted0 and face0 and result: [5.9113274]
compairing completed
Detecting Face...	completed
Calculating Embeddings...	completed
compairing...
compairing wan

Detecting Face...	completed
Calculating Embeddings...	completed
compairing...
compairing wanted0 and face0 and result: [16.353546]
compairing completed
Detecting Face...	completed
Calculating Embeddings...	completed
compairing...
compairing wanted0 and face0 and result: [16.233118]
compairing completed
Detecting Face...	completed
Calculating Embeddings...	completed
compairing...
compairing wanted0 and face0 and result: [16.15462]
compairing completed
Detecting Face...	completed
Calculating Embeddings...	completed
compairing...
compairing wanted0 and face0 and result: [16.567638]
compairing completed
Detecting Face...	completed
Calculating Embeddings...	completed
compairing...
compairing wanted0 and face0 and result: [16.288248]
compairing completed
Detecting Face...	completed
Calculating Embeddings...	completed
compairing...
compairing wanted0 and face0 and result: [15.997224]
compairing completed
Detecting Face...	completed
Calculating Embeddings...	completed
compairing...
compairing 

In [ ]:
cam.release()

In [ ]:
cam = cv2.VideoCapture(0)

while 1:
    ok, frame = cam.read()
    if ok:
        faces = d.face_detector.detect_faces(frame)
        for face in faces:
            [x,y,w,h] = face['box']
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
        cv2.imshow("live feed", frame)
        k = cv2.waitKey(2)
        if k & 0xff == ord('q'):
            cv2.destroyAllWindows()
            cam.release()
            break
    else:
        print('camera not working')